# 1단계: 환경 설정

In [ ]:
!pip install requests pymongo voyageai dnspython langchain langchain-aws langchain-openai tqdm Pillow

from pymongo import MongoClient

# MongoDB 연결 문자열을 넣습니다.
MONGODB_URI = ""
# Voyage AI API Key를 넣습니다.
VOYAGE_API_KEY = ""

MONGODB_URI = ""
VOYAGE_API_KEY = ""

# 데이터베이스 이름
DB_NAME = "mdb_ai_workshop"

# MongoDB Python 클라이언트를 초기화합니다.
mongodb_client = MongoClient(MONGODB_URI)
# 서버 연결 상태를 확인합니다.
mongodb_client.admin.command("ping")

 ## 유틸리티 함수 정의

In [ ]:
from pymongo.collection import Collection
from pymongo.errors import OperationFailure
from typing import List, Dict, Optional
from tqdm import tqdm
from typing import Dict
import time

SLEEP_TIMER = 3

def create_index(collection: Collection, index_name: str, model: Dict) -> None:
    try:
        print(f"Creating the {index_name} index")
        collection.create_search_index(model=model)
    except OperationFailure:
        print(f"{index_name} index already exists, recreating...")
        try:
            print(f"Dropping {index_name} index")
            collection.drop_search_index(name=index_name)

            # Poll for index deletion to complete
            while True:
                indexes = list(collection.list_search_indexes())
                index_exists = any(idx.get("name") == index_name for idx in indexes)
                if not index_exists:
                    print(f"{index_name} index deletion complete")
                    break
                print(f"Waiting for {index_name} index deletion to complete...")
                time.sleep(SLEEP_TIMER)

            print(f"Creating new {index_name} index")
            collection.create_search_index(model=model)
            print(f"Successfully recreated the {index_name} index")
        except Exception as e:
            raise Exception(f"Error during index recreation: {str(e)}")


def check_index_ready(collection: Collection, index_name: str) -> None:
    while True:
        indexes = list(collection.list_search_indexes())
        matching_indexes = [idx for idx in indexes if idx.get("name") == index_name]

        if not matching_indexes:
            print(f"{index_name} index not found")
            time.sleep(SLEEP_TIMER)
            continue

        index = matching_indexes[0]
        status = index["status"]
        if status == "READY":
            print(f"{index_name} index status: READY")
            print(f"{index_name} index definition: {index['latestDefinition']}")
            break

        print(f"{index_name} index status: {status}")
        time.sleep(SLEEP_TIMER)

# 2단계: MongoDB로 데이터 가져오기

##### 아래 변수중 **DB_NAME**을 자신의 이메일 앞자리로 변경합니다

In [ ]:
# 컬렉션 이름
COLLECTION_NAME = "books"
# 벡터 검색 인덱스 이름
ATLAS_SEARCH_INDEX_NAME = f"{DB_NAME}_search_index"
VECTOR_SEARCH_INDEX_NAME = f"{DB_NAME}_vector_search_index"
VECTOR_SEARCH_INDEX_NAME_PRE_FILTER = f"{DB_NAME}_vector_search_index_pre_filter"
VECTOR_SEARCH_INDEX_NAME_QUANTIZED = f"{DB_NAME}_vector_search_index_quantized"

collection = mongodb_client[DB_NAME][COLLECTION_NAME]

In [ ]:
import json

with open("./assets/datas/books.json", "r") as data_file:
    json_data = data_file.read()

data = json.loads(json_data)

print(f"{COLLECTION_NAME} 컬렉션에서 기존 문서를 삭제합니다.")
collection.delete_many({})
collection.insert_many(data)
print(
    f"{collection.count_documents({})}개의 문서가 {COLLECTION_NAME} 컬렉션에 입력되었습니다."
)

# 3단계: Search Index 생성

In [ ]:
search_model = {
    "name": ATLAS_SEARCH_INDEX_NAME,
    "type": "search",
    "definition": {
        "mappings": {"dynamic": False, "fields": {"title": {"type": "string"}}}
    },
}

In [ ]:
# `create_index` 함수를 사용하여 위 정의대로 벡터 검색 인덱스를 생성합니다.
create_index(collection, ATLAS_SEARCH_INDEX_NAME, search_model)

In [ ]:
# 진행하기 전에 `check_index_ready` 함수를 사용하여 인덱스가 생성되었고 'READY' 상태인지 확인합니다.
check_index_ready(ATLAS_SEARCH_INDEX_NAME, collection, VECTOR_SEARCH_INDEX_NAME)

# 4단계: 검색 쿼리 수행하기

https://www.mongodb.com/docs/atlas/atlas-search/operators-collectors/text/#basic-example 참고

In [ ]:
from PIL import Image
import requests

# 검색을 사용하여 사용자 쿼리에 대한 관련 문서를 검색하는 함수를 정의합니다.
def search(
    user_query: str
) -> None:
    pipeline = [
        {
            "$search": {
                "index": ATLAS_SEARCH_INDEX_NAME,
                "text": {"query": user_query, "path": "title"},
            }
        },
        {"$project": {"_id": 0, "title": 1, "cover": 1, "year":1, "pages":1, "score": {"$meta": "vectorSearchScore"}}}
    ] # <CODE_BLOCK>

    # Aggregation `pipeline`을 실행하고 결과를 `results`에 저장합니다.
    results = collection.aggregate(pipeline) # <CODE_BLOCK>

    # 책 제목, 점수, 표지 이미지를 출력합니다.
    for book in results:
        cover = Image.open(requests.get(book.get("cover"), stream=True).raw).resize((100,150))
        print(f"{book.get('title')}({book.get('year')}, {book.get('pages')} pages): {book.get('score')}")
        display(cover)

In [ ]:
# 텍스트 쿼리로 검색을 테스트합니다.

search("sea")

# 5단계: 임베딩 생성

In [ ]:
import voyageai
from PIL import Image

# Voyage AI 클라이언트를 초기화합니다.
vo = voyageai.Client(api_key=VOYAGE_API_KEY)

### 텍스트 임베딩 실습
https://docs.voyageai.com/docs/multimodal-embeddings#python-api (예제 참고)

In [ ]:
text = "Puppy Preschool: Raising Your Puppy Right---Right from the Start!"
# 텍스트 데이터를 임베딩하기 위해 `multimodal_embed` 메서드를 사용합니다.
embedding = vo.multimodal_embed(inputs=[[text]], model="voyage-multimodal-3")

embedding.embeddings[0]

### 이미지 임베딩 실습

In [ ]:
image_url = "https://images.isbndb.com/covers/4318463482198.jpg"
# 위 URL에서 이미지를 불러옵니다.
image = Image.open(requests.get(image_url, stream=True).raw)
# Voyage AI API의 `multimodal_embed` 메서드를 사용하여 이미지를 임베딩합니다:
# inputs: 리스트의 리스트 형태로 이미지를 감싸서 전달
# model: `voyage-multimodal-3` 사용

embedding = vo.multimodal_embed(inputs=[[image]], model="voyage-multimodal-3") #<CODE_BLOCK>

# 객체에서 임베딩 결과(리스트)를 가져옵니다.
embedding.embeddings[0]

# 6단계: MongoDB의 기존 데이터에 임베딩 추가하기

In [ ]:
# 임베딩할 문서의 필드 (여기서는 책 표지 이미지)
field_to_embed = "cover"
# 문서에 추가될 임베딩 필드의 이름
embedding_field = "embedding"

In [ ]:
def get_embeddings(content: str, mode: str, input_type: str) -> List[float]:
    # 입력이 이미지인 경우, 먼저 이미지 콘텐츠를 로드합니다.
    if mode == "image":
        if content.startswith("http"):
            content = Image.open(requests.get(content, stream=True).raw)
        else:
            content = Image.open(content)
    return vo.multimodal_embed(inputs=[[content]], model="voyage-multimodal-3", input_type=input_type).embeddings[0]

https://www.mongodb.com/docs/manual/tutorial/query-documents/#select-all-documents-in-a-collection

In [ ]:
# `collection` 컬렉션의 모든 문서를 조회합니다.
results = collection.find({}) # <CODE_BLOCK>

**$set:** https://www.mongodb.com/docs/manual/reference/operator/update/set/#syntax

**update_one():** https://www.mongodb.com/docs/manual/reference/method/db.collection.updateOne/#definition

In [ ]:
# 컬렉션 내 각 문서에 임베딩을 추가합니다.
for result in tqdm(results):
    content = result[field_to_embed]
    # 위에서 정의한 `get_embeddings` 함수를 사용하여 `content`를 임베딩합니다.
    # `content`는 책 표지 이미지이므로 `mode`를 그에 맞게 설정하세요.
    # 검색 대상인 "문서"를 임베딩하므로 `input_type`은 "document"로 설정해야 합니다.
    embedding = get_embeddings(content, "image", "document") #<CODE_BLOCK>
    # 현재 문서의 `_id`와 일치하는 문서를 필터링합니다.
    filter = {"_id": result["_id"]}
    # `$set` 연산자를 사용하여 `embedding_field` 필드에 `embedding` 값을 설정합니다.
    update = {"$set": {embedding_field: embedding}} # <CODE_BLOCK>
    # `update_one()` 작업을 사용하여 컬렉션 내 문서를 제자리(inplace) 업데이트합니다.
    # `filter`를 사용하여 올바른 문서 `_id`를 찾고 `update`를 적용하세요.
    collection.update_one(filter, update) # <CODE_BLOCK>

# 7단계: 벡터 검색 인덱스 생성

In [ ]:
# 벡터 인덱스 정의를 생성합니다. 다음 항목들을 지정하세요:
# path: 임베딩 필드의 경로
# numDimensions: 임베딩 차원 수 (사용된 임베딩 모델에 따라 다름)
# similarity: 유사도 측정 방식 (cosine, euclidean, dotProduct 중 하나)
vector_search_model = {
    "name": VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 1024,
                "similarity": "cosine",
            }
        ]
    },
}

In [ ]:
# `create_index` 함수를 사용하여 위 정의대로 벡터 검색 인덱스를 생성합니다.
create_index(collection, VECTOR_SEARCH_INDEX_NAME, vector_search_model)

In [ ]:
# 진행하기 전에 `check_index_ready` 함수를 사용하여 인덱스가 생성되었고 'READY' 상태인지 확인합니다.
check_index_ready(collection, VECTOR_SEARCH_INDEX_NAME)

# 8단계: 벡터 검색 쿼리 수행하기

https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#ann-examples ("Filter Example" 참고)

In [ ]:
# 벡터 검색을 사용하여 사용자 쿼리에 대한 관련 문서를 검색하는 함수를 정의합니다.
def vector_search(
    index_name: str, user_query: str, mode: str, filter: Optional[Dict] = {}
) -> None:
    # 4단계에서 정의한 `get_embeddings` 함수를 사용하여 `user_query`에 대한 임베딩을 생성합니다.
    # 쿼리를 임베딩하므로 `input_type`은 "query"로 설정해야 합니다.
    query_embedding = get_embeddings(user_query, mode, "query") #<CODE_BLOCK>

    # $vectorSearch 스테이지와 $project 스테이지로 구성된 Aggregation 파이프라인을 정의합니다.
    # 후보(candidates) 수는 20개로 설정하고, 벡터 검색 결과 상위 5개 문서만 반환합니다.
    # $vectorSearch 스테이지의 `filter` 필드에 함수로 전달된 `filter` 값을 설정합니다.
    # $project 스테이지에서는 `_id` 필드를 제외하고, `title`, `cover`, `year`, `pages`, 그리고 `vectorSearchScore`를 포함합니다.
    # 참고: $vectorSearch 스테이지의 `index`, `queryVector`, `path` 필드에는 이전에 정의한 변수를 사용하세요.
    pipeline = [
        {
            "$vectorSearch": {
                "index": index_name,
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 20,
                "filter": filter,
                "limit": 5,
            }
        },
        {"$project": {"_id": 0, "title": 1, "cover": 1, "year":1, "pages":1, "score": {"$meta": "vectorSearchScore"}}}
    ] # <CODE_BLOCK>

    # Aggregation `pipeline`을 실행하고 결과를 `results`에 저장합니다.
    results = collection.aggregate(pipeline) # <CODE_BLOCK>

    # 책 제목, 점수, 표지 이미지를 출력합니다.
    for book in results:
        cover = Image.open(requests.get(book.get("cover"), stream=True).raw).resize((100,150))
        print(f"{book.get('title')}({book.get('year')}, {book.get('pages')} pages): {book.get('score')}")
        display(cover)

In [ ]:
# 텍스트 쿼리로 벡터 검색을 테스트합니다.
vector_search(VECTOR_SEARCH_INDEX_NAME, "sea", "text")

# 다음 텍스트 쿼리들도 시도해 보세요:
# - A man wearing a golden crown
# - A rainbow of lively colors
# - Creatures wondrous or familiar
# - A boy and the ocean
# - Houses

In [ ]:
# 이미지 쿼리로 벡터 검색을 테스트합니다.
vector_search(VECTOR_SEARCH_INDEX_NAME, "https://images.isbndb.com/covers/20721003482321.jpg", "image")

# 다음 이미지 쿼리들도 시도해 보세요:
# - ../data/images/salad.jpg
# - ../data/images/kitten.png
# - ../data/images/barn.png

# 9단계: 벡터 검색에 사전 필터(Pre-filters) 추가하기

### `2002`년 이후 출판되었고, 페이지 수가 `250`쪽 이하인 책 필터링하기

https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#about-the-filter-type

In [ ]:
# 5단계의 벡터 검색 인덱스 `model`을 수정하여 `year` 필드를 `filter` 필드로 추가합니다.
vector_search_per_filter_model = {
    "name": VECTOR_SEARCH_INDEX_NAME_PRE_FILTER,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 1024,
                "similarity": "cosine",
            },
            {"type": "filter", "path": "year"},
            {"type": "filter", "path": "pages"},
        ]
    }
}

In [ ]:
# `create_index` 함수를 사용하여 수정된 모델로 벡터 검색 인덱스를 재생성합니다.
create_index(collection, vector_search_per_filter_model)

In [ ]:
# `check_index_ready` 함수를 사용하여 인덱스에 올바른 필터 필드가 적용되었고 'READY' 상태인지 확인합니다.
check_index_ready(collection, VECTOR_SEARCH_INDEX_NAME_PRE_FILTER)

https://www.mongodb.com/docs/manual/reference/operator/query/gte/#syntax
https://www.mongodb.com/docs/manual/reference/operator/query/lte/#mongodb-query-op.-lte

In [ ]:
# `year` 필드가 `2002` 이상이고(`$gte`), `pages` 필드가 `250` 이하인(`$lte`) 책을 필터링하는 정의를 생성합니다.
filter = {"$and": [{"year": {"$gte": 2002}}, {"pages": {"$lte": 250}}]} # <CODE_BLOCK>
# `filter`를 `vector_search` 함수의 인자로 전달합니다.
# 이 필터가 `vector_search` 함수 내의 `pipeline`에 어떻게 통합되는지 확인해 보세요.
vector_search(VECTOR_SEARCH_INDEX_NAME_PRE_FILTER, "A boy and the ocean", "text", filter)

# 10단계: 벡터 양자화(Vector Quantization) 활성화 (skip)

https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/#syntax

In [ ]:
# 5단계의 벡터 검색 인덱스 `model`을 수정하여 `scalar` 양자화를 사용하도록 설정합니다.
vector_search_quantized_model = {
    "name": VECTOR_SEARCH_INDEX_NAME_QUANTIZED,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 1024,
                "similarity": "cosine",
                "quantization": "scalar",
            },
        ]
    },
}

In [ ]:
# `utils` 모듈의 `create_index` 함수를 사용하여 수정된 모델로 벡터 검색 인덱스를 재생성합니다.
create_index(collection, VECTOR_SEARCH_INDEX_NAME_QUANTIZED, vector_search_quantized_model)

In [ ]:
# `check_index_ready` 함수를 사용하여 양자화가 활성화된 상태로 인덱스가 생성되었는지 확인합니다.
check_index_ready(collection, VECTOR_SEARCH_INDEX_NAME_QUANTIZED)

# 11단계: Hybrid 검색 수행

**참고:** 아래 하이브리드 검색 쿼리는 검색과 벡터 검색을 수동으로 조합하는 방식입니다.

$rankFusion 이라는 새로운 Aggreation Stage가 Preview로 제공되고있습니다

https://www.mongodb.com/docs/atlas/atlas-vector-search/hybrid-search/vector-search-with-full-text-search/#run-a-combined-semantic-search-and-full-text-search-query-using--rankfusion

In [ ]:
def hybrid_search(
    user_query: str, vector_weight: float, full_text_weight: float
) -> None:
    """
    하이브리드 검색을 사용하여 사용자 쿼리에 대한 관련 문서를 검색합니다.

    Args:
        user_query (str): 사용자 쿼리 문자열
        vector_weight (float): 최종 검색 결과에서 벡터 검색이 차지하는 가중치
        full_text_weight (float): 최종 검색 결과에서 전문 검색이 차지하는 가중치
    """
    pipeline = [
        # 벡터 검색(Vector search) 스테이지
        {
            "$vectorSearch": {
                "index": VECTOR_SEARCH_INDEX_NAME,
                "path": "embedding",
                "queryVector": get_embeddings(user_query, "text", "query"),
                "numCandidates": 20,
                "limit": 10,
            }
        },
        # 벡터 검색으로 반환된 모든 문서를 'docs'라는 하나의 배열로 그룹화합니다.
        {"$group": {"_id": None, "docs": {"$push": "$$ROOT"}}},
        # 'docs' 필드의 문서 배열을 풀고(unwind), 배열 내 문서의 위치(인덱스)를 'rank'라는 필드에 저장합니다.
        {"$unwind": {"path": "$docs", "includeArrayIndex": "rank"}},
        # 'docs' 배열의 각 문서에 대해 Reciprocal Rank(상호 순위) 점수를 계산하여 'vs_score' 필드에 추가합니다.
        {
            "$addFields": {
                "vs_score": {
                    "$multiply": [
                        vector_weight,
                        {"$divide": [1.0, {"$add": ["$rank", 60]}]},
                    ]
                }
            }
        },
        # 결과에 'vs_score', '_id', 'title', 'cover' 필드만 포함합니다.
        {
            "$project": {
                "vs_score": 1,
                "_id": "$docs._id",
                "title": "$docs.title",
                "cover": "$docs.cover",
            }
        },
        # 전문 검색(Full-text search) 스테이지와 결합(Union)합니다.
        {
            "$unionWith": {
                "coll": COLLECTION_NAME,
                "pipeline": [
                    # 전문 검색(Full-text search)
                    {
                        "$search": {
                            "index": ATLAS_SEARCH_INDEX_NAME,
                            "text": {"query": user_query, "path": "title"},
                        }
                    },
                    # 전문 검색 결과를 10개로 제한합니다.
                    {"$limit": 10},
                    # 전문 검색으로 반환된 모든 문서를 'docs'라는 하나의 배열로 그룹화합니다.
                    {"$group": {"_id": None, "docs": {"$push": "$$ROOT"}}},
                    # 'docs' 필드의 문서 배열을 풀고(unwind), 배열 내 문서의 위치(인덱스)를 'rank' 필드에 저장합니다.
                    {"$unwind": {"path": "$docs", "includeArrayIndex": "rank"}},
                    # 'docs' 배열의 각 문서에 대해 Reciprocal Rank(상호 순위) 점수를 계산하여 'fts_score' 필드에 추가합니다.
                    {
                        "$addFields": {
                            "fts_score": {
                                "$multiply": [
                                    full_text_weight,
                                    {"$divide": [1.0, {"$add": ["$rank", 60]}]},
                                ]
                            }
                        }
                    },
                    # 결과에 'fts_score', '_id', 'title', 'cover' 필드만 포함합니다.
                    {
                        "$project": {
                            "fts_score": 1,
                            "_id": "$docs._id",
                            "title": "$docs.title",
                            "cover": "$docs.cover",
                        }
                    },
                ],
            }
        },
        # 다음 스테이지들은 벡터 검색과 전문 검색 결과를 하나의 순위 목록으로 결합합니다.
        # 결과를 `title`, `vs_score`, `fts_score` 기준으로 그룹화합니다.
        {
            "$group": {
                "_id": "$_id",
                "title": {"$first": "$title"},
                "vs_score": {"$max": "$vs_score"},
                "fts_score": {"$max": "$fts_score"},
                "cover": {"$first": "$cover"},
            }
        },
        # 결과에 `_id`, `title`, `cover`, `vs_score`, `fts_score` 필드만 포함합니다.
        {
            "$project": {
                "_id": 1,
                "title": 1,
                "vs_score": {"$ifNull": ["$vs_score", 0]},
                "fts_score": {"$ifNull": ["$fts_score", 0]},
                "cover": 1,
            }
        },
        # `vs_score`와 `fts_score`의 합계를 포함하는 `score` 필드를 추가합니다.
        {
            "$project": {
                "score": {"$add": ["$fts_score", "$vs_score"]},
                "_id": 1,
                "title": 1,
                "vs_score": 1,
                "fts_score": 1,
                "cover": 1,
            }
        },
        # `score`를 기준으로 내림차순 정렬합니다.
        {"$sort": {"score": -1}},
        # 결과를 상위 5개로 제한합니다.
        {"$limit": 5},
    ]

    results = collection.aggregate(pipeline)

    # Print book title, scores, and cover image
    for book in results:
        cover = Image.open(requests.get(book["cover"], stream=True).raw).resize(
            (100, 150)
        )
        print(
            f"{book.get('title')}, VS Score: {book.get('vs_score')}, FTS Score: {book.get('fts_score')}"
        )
        display(cover)

In [ ]:
# 하이브리드 검색을 테스트합니다. (벡터 검색 가중치 1.0, 전문 검색 가중치 0.0)
hybrid_search(
    user_query="My Favorite Summer",
    vector_weight=8.0,
    full_text_weight=2.0,
)

In [ ]:
# 하이브리드 검색을 테스트합니다. (벡터 검색 가중치 0.3, 전문 검색 가중치 0.7)
hybrid_search(
    user_query="My Favorite Summer",
    vector_weight=0.3,
    full_text_weight=0.7,
)